# Projeto para Turismo - Planejador de Viagens (Multiagente)



## Instalação das bibliotecas




In [1]:
!pip install -q crewai[tools]==0.120.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.0/310.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.7/602.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.4 MB/s

## Carregamento do modelo



In [2]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = "#######"

In [49]:
from crewai import LLM

id_model = "gemini/gemini-2.5-flash" # @param {type:"string"}

gemini = LLM(
    model = id_model,
    api_key=os.getenv("GOOGLE_API_KEY")
)

In [50]:
## Apenas para testar o modelo
#resposta = gemini.call("Me diga uma curiosidade sobre o espaço")
#resposta

'Uma curiosidade fascinante sobre o espaço é que, apesar de ser amplamente conhecido como um vácuo, o espaço interestelar não é completamente vazio. Ele contém uma quantidade extremamente baixa de partículas, principalmente hidrogênio e hélio, além de poeira cósmica. Essa matéria, embora esparsa, pode se agregar ao longo de milhões ou bilhões de anos para formar novas estrelas e planetas.\n'

## Criação de Tools (ferramentas)



In [4]:
!pip install -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.4 MB/s eta 0:00:00


In [5]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults

In [6]:
@tool
def search_web_tool(query: str):
  """
  Searches the web and returns results.
  """
  search_tool = DuckDuckGoSearchResults(num_results=4, verbose=True)
  return search_tool.run(query)

In [8]:
#from langchain_community.tools import DuckDuckGoSearchRun
#search = DuckDuckGoSearchRun()
#search.invoke("eventos em londres no próximo mês")

### Criação de classe



In [9]:
!pip install -q langchain-tavily

In [10]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


In [11]:
from langchain_tavily import TavilySearch
from langchain_community.tools import DuckDuckGoSearchResults

class SearchTools:
  @tool("Pesquisa na internet")
  def search_tavily(query: str = "") -> str:
      """
      Search the web using Tavily API.
      Recommended for more structured and recent information.
      """
      search_tavily = TavilySearch(max_results=4)
      search_res = search_tavily.invoke(query)
      return search_res

  @tool("Pesquisa na internet com DuckDuckGo")
  def search_duckduckgo(query: str):
      """
      Search the web using DuckDuckGo.
      Returns a list of search results.
      """
      search_tool = DuckDuckGoSearchResults(num_results=4, verbose=True)
      return search_tool.run(query)

In [12]:
SearchTools.search_tavily

Tool(name='Pesquisa na internet', description="Tool Name: Pesquisa na internet\nTool Arguments: {'query': {'description': None, 'type': 'str'}}\nTool Description: \n      Search the web using Tavily API.\n      Recommended for more structured and recent information.\n      ", args_schema=<class 'abc.Pesquisanainternet'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7fb3d100ad40>, result_as_answer=False, func=<function SearchTools.search_tavily at 0x7fb3c74a6fc0>)

### Outras ferramentas



In [13]:
class CalculatorTools:
    @tool("Faça um cálculo")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Erro: Sintaxe inválida"

CalculatorTools.calculate

Tool(name='Faça um cálculo', description='Tool Name: Faça um cálculo\nTool Arguments: {}\nTool Description: Useful to perform any mathematical calculations,\n        like sum, minus, multiplication, division, etc.\n        The input to this tool should be a mathematical\n        expression, a couple examples are `200*7` or `5000/2*10`\n        ', args_schema=<class 'abc.Façaumcálculo'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7fb3d100ad40>, result_as_answer=False, func=<function CalculatorTools.calculate at 0x7fb3c68b8ea0>)


## Criação de Agentes






In [14]:
from textwrap import dedent

texto = """
        Este é um texto
        com indentação errada.
"""
print(texto)


        Este é um texto
        com indentação errada.



In [15]:
texto = dedent("""
        Este é um texto
        com indentação corrigida.
    """)
print(texto)


Este é um texto
com indentação corrigida.



### Especialista em informações da cidade



In [30]:
from crewai import Agent

city_info_agent = Agent(
    role="Especialista em informações da cidade",
    goal=dedent(
        """Reunir informações úteis sobre o destino escolhido, com base nos interesses e datas da viagem,
        ajudando viajantes a entender o contexto geral do local."""
    ),
    backstory=dedent(
        """
        Sou um especialista com amplo conhecimento sobre cidades ao redor do mundo,
        capaz de fornecer detalhes atualizados sobre clima, eventos culturais, segurança,
        costumes locais e outros aspectos práticos para quem pretende visitar o local.
        Sou apaixonado por compartilhar as melhores experiências e 'joias escondidas' do local.
        """
    ),
    llm=gemini,
    tools=[SearchTools.search_tavily],
    verbose=True,
    max_iter=5,
    allow_delegation=False,
)

### Logística e informações da cidade



In [31]:
logistics_expert_agent = Agent(
    role="Especialista em logística de viagem",
    goal="Identificar as melhores opções logísticas para a viagem, com foco em praticidade, custo-benefício e conforto.",
    backstory=dedent(
        """
        Sou um profissional focado em planejar a logística da viagem: transporte, hospedagem e deslocamento local.
        Tenho conhecimento sobre companhias aéreas, apps de mobilidade, regiões seguras para se hospedar e otimização de trajetos.
        """
    ),
    llm=gemini,
    tools=[
        SearchTools.search_tavily,
        CalculatorTools.calculate
    ],
    verbose=True,
    max_iter=5,
    allow_delegation=False,
)

### Especialista em criação de itinerário



In [32]:
itinerary_planner_agent = Agent(
    role="Planejador de itinerário personalizado",
    goal="Criar um roteiro completo com base nas preferências do usuário.",
    backstory=dedent(
        """
        Sou um guia profissional apaixonado por viagens, especialista em organização de itinerários personalizados.
        Minha missão é integrar dados sobre clima, atrações, eventos e logística em uma experiência otimizada para o turista.
        """
    ),
    llm=gemini,
    tools=[SearchTools.search_tavily],
    verbose=True,
    max_iter=5,
    allow_delegation=False,
)

## Criação das Tarefas (Tasks)



In [33]:
from crewai import Task

from_city = "São Paulo" # @param {type: "string"}
destination_city = "Londres" # @param {type: "string"}
date_from = "23 de novembro de 2025" # @param {type: "string"}
date_to = "30 de novembro de 2025" # @param {type: "string"}
interests = "arte, tecnologia, vinhos, arquitetura" # @param {type: "string"}

print(f"""
Viajando de: {from_city}
Para: {destination_city}
Chegada: {date_from}
Partida: {date_to}
""")


Viajando de: São Paulo
Para: Londres
Chegada: 23 de novembro de 2025
Partida: 30 de novembro de 2025



### Task: Coleta informações da cidade



In [34]:
def city_info_task(agent, from_city, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes locais.
            Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e quaisquer atividades que se alinhem às preferências do usuário.
            Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.
            Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.

            Viajando de: {from_city}
            Para: {destination_city}
            Interesses do viajante: {interests}
            Chegada: {date_from}
            Partida: {date_to}

            Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.
            """
        ),
        expected_output=dedent(
            f"""
            Um guia detalhado (em formato markdown) em português que inclui:
            - Resumo da cidade e sua cultura;
            - Uma lista selecionada de lugares recomendados para visitar e eventos (se houver);
            - Um detalhamento das despesas diárias, como custos médios com alimentação;
            - Recomendações de segurança;
            - Dicas de costumes locais;
            """
        ),
        agent=agent,
        output_file='relatorio_local.md',
    )

### Task: Planejamento de logística



In [35]:
def plan_logistics_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Planejar a logística da viagem.
            Identificar as melhores opções de hospedagem, voos e transporte local considerando:

            Destino: {destination_city}
            Data de chegada: {date_from}
            Data de partida: {date_to}
            Interesses: {interests}

            Avalie preço, localização, conveniência e segurança.
            """
        ),
        expected_output=dedent(
            f"""
            Relatório em português (em formato markdown):
            - Sugestão de hospedagens, preferencialmente com localização estratégica;
            - Opções de voos ou meios de transporte para chegada/partida;
            - Sugestões de deslocamento dentro da cidade;
            - Estimativas de custo para cada item;
            """
        ),
        context=context,
        agent=agent,
        output_file='relatorio_logistica.md',
    )

### Task: Construção itinerário final



In [36]:
def build_itinerary_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Esta tarefa sintetiza todas as informações para criar o roteiro final da viagem.
            Com base nas informações dos outros agentes, desenvolva um itinerário detalhado.
            Cada dia deve conter atividades, clima, transporte, refeições e estimativa de gastos.

            Destino: {destination_city}
            Interesses: {interests}
            Data de chegada: {date_from}
            Data de partida: {date_to}
            """
        ),
        expected_output=dedent("""
        Documento em português que inclui (em formato markdown):
        - Boas vindas e apresentação da cidade em até 2 parágrafos.
        - Programação diária com sugestões de horários e atividades
        - Atrações distribuídas por região e logística
        - Clima previsto, sugestões de transporte e alimentação
        - Eventos e festivais sazonais (se houver)
        - Estimativa de gastos por dia, custo médio das despesas diárias e pontos turísticos.
        - Visão geral dos destaques da cidade com base nas recomendações do guia.
        - Outras dicas e informações adicionais para uma viagem e estadia tranquila.
        """),
        context=context,
        agent=agent,
        output_file='roteiro_viagem.md',
    )

### Executando a criação das tasks



In [37]:
city_info = city_info_task(
  city_info_agent,
  from_city,
  destination_city,
  interests,
  date_from,
  date_to
)

In [38]:
plan_logistics = plan_logistics_task(
  [city_info],
  logistics_expert_agent,
  destination_city,
  interests,
  date_from,
  date_to
)

In [39]:
build_itinerary = build_itinerary_task(
  [city_info, plan_logistics],
  itinerary_planner_agent,
  destination_city,
  interests,
  date_from,
  date_to,
)

## Criação da equipe (Crew)



In [44]:
from crewai import Crew, Process

crew = Crew(
  agents=[city_info_agent, logistics_expert_agent, itinerary_planner_agent],
  tasks=[city_info, plan_logistics, build_itinerary],
  process=Process.sequential,
  full_output=True,
  share_crew=False,
  verbose=True,
  max_rpm=10
)
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3f09e65e-2c16-44f9-9a82-e63a1dd54d96                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a0eb1d41-0dd3-4874-a9b6-9f3e81a87406
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a0eb1d41-0dd3-4874-a9b6-9f3e81a87406
       Status: Executing Task...
    └── 🤖 Agent: Especialista em informações da cidade
            Status: In Progress

# Agent: Especialista em informações da cidade
## Task: 
Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes locais.
Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e quaisquer atividades que se alinhem às preferências do usuário.
Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.
Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.

Viajando de: São Paulo
Para: Londres
Interesses do viajante: arte, tecnologia, vinhos, arquitetura
Chegada: 23 de novembro de 2025
Partida: 30 de novembro de 2025

Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.



🤖 Agent: Especialista em informações da cidade
    Status: In Progress

...

 
[2025-09-27 11:32:05][INFO]: Max RPM reached, waiting for next minute to start.


# Agent: Planejador de itinerário personalizado
## Final Answer: 
# Roteiro Personalizado: Londres - Arte, Tecnologia, Vinhos e Arquitetura

Bem-vindo(a) a Londres! Prepare-se para uma semana inesquecível na capital britânica, uma metrópole vibrante que harmoniza perfeitamente a rica história com a vanguarda da tecnologia e da cultura. Das grandiosas galerias de arte aos pubs aconchegantes, passando por inovações arquitetônicas e experiências gastronômicas únicas, Londres oferece um cenário perfeito para explorar seus interesses em arte, tecnologia, vinhos e arquitetura.

Este roteiro foi cuidadosamente elaborado para otimizar sua experiência, considerando as datas da sua viagem (23 a 30 de novembro de 2025) e seus interesses. Navegaremos pela cidade de forma eficiente, agrupando atrações por região e garantindo que você aproveite ao máximo cada momento, mesmo com o clima mais frio e os dias mais curtos

🚀 Crew: crew
├── 📋 Task: a0eb1d41-0dd3-4874-a9b6-9f3e81a87406
│      Assigned to: Especialista em informações da cidade
│      Status: ✅ Completed
│   ├── 🤖 Agent: Especialista em informações da cidade
│   │       Status: In Progress
│   │   └── 🔧 Used Pesquisa na internet (26)
│   └── 🤖 Agent: Especialista em informações da cidade
│           Status: ✅ Completed
├── 📋 Task: 77268429-168d-494f-b99f-32a3d6e693a9
│      Assigned to: Especialista em logística de viagem
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em logística de viagem
│           Status: ✅ Completed
└── 📋 Task: adda3ad9-45e7-4aad-9b02-87cccabd82de
       Status: Executing Task...
    └── 🤖 Agent: Planejador de itinerário personalizado
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: a0eb1d41-0dd3-4874-a9b6-9f3e81a87406
│      Assigned to: Especialista em informações da cidade
│      Status: ✅ Completed
│   ├── 🤖 Agent: Especialista em informações da cidade
│   │       Status: In Progress
│   │   └── 🔧 Used Pesquisa na internet (26)
│   └── 🤖 Agent: Especialista em informações da cidade
│           Status: ✅ Completed
├── 📋 Task: 77268429-168d-494f-b99f-32a3d6e693a9
│      Assigned to: Especialista em logística de viagem
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em logística de viagem
│           Status: ✅ Completed
└── 📋 Task: adda3ad9-45e7-4aad-9b02-87cccabd82de
       Assigned to: Planejador de itinerário personalizado
       Status: ✅ Completed
    └── 🤖 Agent: Planejador de itinerário personalizado
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: adda3ad9-45e7-4aad-9b02-87cccabd82de                                                                     │
│  Agent: Planejador de itinerário personalizado                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3f09e65e-2c16-44f9-9a82-e63a1dd54d96                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Visualizar resultado



In [45]:
from IPython.display import display, Markdown

def load_markdown(file_path):
  try:
    with open(file_path, 'r', encoding='utf-8') as file:
      content = file.read()
      content = content.replace("```markdown", "").replace("```", "")
      return content
  except Exception as e:
    print(f"Erro ao carregar o arquivo: {str(e)}")
    return None

In [46]:
def display_markdown(content):
  return display(Markdown(content))

In [ ]:
md = load_markdown("/content/roteiro_viagem.md")
display_markdown(md)


## Guia de Viagem para Londres (23 a 30 de Novembro de 2025)

Bem-vindo a Londres! Prepare-se para uma semana inesquecível explorando a capital britânica, com foco em arte, tecnologia, vinhos e arquitetura.

### Resumo da Cidade e sua Cultura

Londres é uma metrópole vibrante com uma história rica e uma cultura diversificada. É um centro global de arte, moda, finanças e tecnologia, onde o tradicional encontra o moderno em cada esquina. A cidade abriga alguns dos museus, galerias de arte, marcos históricos e arquitetônicos mais renomados do mundo. Sua cena gastronômica é igualmente diversificada, com restaurantes que oferecem culinária de todos os cantos do globo, além de pubs tradicionais e wine bars sofisticados.

### Hospedagem

**Sugestões de Hospedagem:**

*   **The Henrietta Hotel (Covent Garden):** Hotel boutique estiloso no coração do West End, próximo a teatros, restaurantes e atrações turísticas. Ideal para quem busca uma experiência charmosa e conveniente. Estimativa de custo: £285 por noite (pesquisado em 25/05/2025 para datas futuras, pode variar).
*   **The Rockwell Hotel (Kensington):** Hotel boutique sofisticado em Kensington, próximo a museus, parques e lojas. Ideal para quem busca uma localização mais tranquila e elegante. Estimativa de custo: £250 por noite (estimativa).

**Localização Estratégica:**

Ambos os hotéis estão localizados em áreas seguras e bem conectadas por transporte público, facilitando o acesso às principais atrações de Londres. Covent Garden é ideal para quem busca estar no centro da agitação, enquanto Kensington oferece uma atmosfera mais relaxante.

### Voos

**Opções de Voos (JFK - LHR):**

*   **Companhias Aéreas:** Virgin Atlantic, American Airlines, British Airways, Delta Air Lines, JetBlue Airways (voos diretos).
*   **Estimativa de Custo:** US$424 - US$539 (ida e volta).
*   **Observação:** Os preços podem variar dependendo da antecedência da reserva, da companhia aérea e da classe tarifária. Recomenda-se pesquisar e reservar com antecedência para obter melhores preços.

### Clima em Novembro

Em novembro, Londres tem um clima frio e úmido. As temperaturas médias variam entre 6°C e 11°C. Há possibilidade de chuva e neblina.

**O que vestir:**

*   Casaco quente e impermeável
*   Cachecol, luvas e gorro
*   Sapatos confortáveis e resistentes à água
*   Agasalhos para se proteger do frio

### Programação Diária Sugerida

**Dia 1 (23 de Novembro): Chegada e Arte Moderna**

*   Manhã: Chegada ao aeroporto de Heathrow (LHR). Traslado para o hotel (The Henrietta Hotel ou The Rockwell Hotel).
*   Tarde: Visita à Tate Modern (arte moderna e contemporânea).
*   Noite: Jantar no Borough Market seguido de um vinho no Gordon's Wine Bar (o wine bar mais antigo de Londres).
*   Transporte: Tube (metrô)
*   Estimativa de gastos: Transporte (£15), Almoço (£25), Jantar e vinho (£50)

**Dia 2 (24 de Novembro): História e Arquitetura**

*   Manhã: Visita à Tower of London (história e joias da coroa).
*   Tarde: Passeio pelo Tower Bridge e visita à St. Paul's Cathedral (arquitetura barroca).
*   Noite: Jantar em um pub tradicional e show no Shakespeare's Globe.
*   Transporte: Tube e caminhada
*   Estimativa de gastos: Atrações (£40), Almoço (£25), Jantar e show (£70)

**Dia 3 (25 de Novembro): Arte Clássica e West End**

*   Manhã: Visita à National Gallery (arte europeia do século XIII ao XIX).
*   Tarde: Exploração da Trafalgar Square e visita ao British Museum (artefatos de todo o mundo).
*   Noite: Jantar e um espetáculo musical no West End.
*   Transporte: Tube e caminhada
*   Estimativa de gastos: Almoço (£25), Jantar e teatro (£90)

**Dia 4 (26 de Novembro): Tecnologia e Inovação**

*   Manhã: Visita ao Science Museum (ciência e tecnologia).
*   Tarde: Exploração de Shoreditch e Silicon Roundabout (startups de tecnologia).
*   Noite: Jantar em um restaurante moderno em Shoreditch.
*   Transporte: Tube e ônibus
*   Estimativa de gastos: Almoço (£30), Jantar (£60)

**Dia 5 (27 de Novembro): Arquitetura Moderna e Vistas Panorâmicas**

*   Manhã: Visita ao Barbican Centre (arquitetura brutalista).
*   Tarde: Subida ao The Shard ou Sky Garden (vistas panorâmicas da cidade).
*   Noite: Degustação de vinhos no Vinoteca.
*   Transporte: Tube e caminhada
*   Estimativa de gastos: Atrações (£35), Almoço (£30), Vinho (£45)

**Dia 6 (28 de Novembro): Viagem a Bletchley Park**

*   Dia inteiro: Excursão a Bletchley Park (história da computação e decifração de códigos).
*   Noite: Jantar em um restaurante local próximo ao hotel.
*   Transporte: Trem (ida e volta)
*   Estimativa de gastos: Trem (£30), Atrações (£30), Almoço e jantar (£70)

**Dia 7 (29 de Novembro): Compras e Relaxamento**

*   Manhã: Compras na Oxford Street e Regent Street.
*   Tarde: Passeio pelo Hyde Park e visita ao Kensington Palace.
*   Noite: Jantar de despedida em um restaurante elegante.
*   Transporte: Tube e caminhada
*   Estimativa de gastos: Almoço (£35), Compras (variável), Jantar (£80)

### Atrações por Região e Logística

*   **Westminster:** Houses of Parliament, Big Ben, Westminster Abbey.
*   **City of London:** St. Paul's Cathedral, Tower of London, Tower Bridge.
*   **South Bank:** Tate Modern, Shakespeare's Globe, Borough Market.
*   **Kensington:** Science Museum, Natural History Museum, Victoria and Albert Museum, Kensington Palace, Hyde Park.
*   **Shoreditch:** Silicon Roundabout, arte de rua, restaurantes modernos.

Use o Tube para se locomover entre as regiões. Caminhe para explorar cada área em detalhes.

### Eventos e Festivais Sazonais

*   **Lord Mayor's Show (Novembro):** Um desfile anual que celebra a eleição do novo Lord Mayor de Londres (verifique a data exata para 2025).

### Estimativa de Custos Totais (por pessoa)

*   **Hospedagem (7 noites):** £1785 - £1995 (considerando £255-£285 por noite)
*   **Voos (ida e volta):** £340 - £430 (considerando câmbio de 1.25)
*   **Transporte (7 dias):** £70 - £140
*   **Alimentação (7 dias):** £350 - £700 (considerando £50-£100 por dia)
*   **Atrações (estimativa):** £140 - £350

**Custo Total Estimado:** £2685 - £3615

**Observações:**

*   Esta é apenas uma estimativa. Os custos reais podem variar dependendo das suas escolhas e do seu estilo de viagem.
*   Muitos museus em Londres são gratuitos, o que pode ajudar a reduzir os custos com atrações.
*   Comprar um London Pass pode ser vantajoso se você pretende visitar muitas atrações pagas.
*   Reserve voos e hospedagem com antecedência para garantir melhores preços e disponibilidade.

### Visão Geral dos Destaques da Cidade

*   **Arte:** Tate Modern, National Gallery, British Museum.
*   **Tecnologia:** Science Museum, Bletchley Park, Silicon Roundabout.
*   **Arquitetura:** Tower of London, St. Paul's Cathedral, The Shard, Barbican Centre.
*   **Vinho:** Gordon's Wine Bar, Vinoteca.

### Outras Dicas e Informações Adicionais

*   **Segurança:** Londres é geralmente segura, mas fique atento aos seus pertences, especialmente em áreas turísticas.
*   **Transporte:** Use um Oyster card ou pagamento contactless para economizar no transporte público.
*   **Clima:** Esteja preparado para mudanças climáticas repentinas e sempre leve um casaco ou guarda-chuva.
*   **Reservas:** Faça reservas para restaurantes populares, shows e atrações com antecedência.
*   **Língua:** Inglês.
*   **Moeda:** Libra esterlina (£).
*   **Tomadas:** Tipo G (adaptador necessário).
*   **Emergências:** Ligue para 999.

Aproveite sua viagem a Londres!


## Converter para PDF



In [ ]:
!pip install -q weasyprint markdown2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.6/850.6 kB 34.9 MB/s eta 0:00:00


In [ ]:
import markdown2
from weasyprint import HTML

def convert_md_to_pdf(file_md, file_pdf):
  text_md = load_markdown(file_md)
  if text_md is not None:
    html_body = markdown2.markdown(text_md)
    style_css = """
    <style>
        body {
            font-family: Arial, Helvetica, sans-serif;
            font-size: 12pt;
            line-height: 1.5;
        }
    </style>
    """
    full_html = f"<html><head>{style_css}</head>{html_body}</html>"
    HTML(string=full_html).write_pdf(file_pdf)

In [ ]:
convert_md_to_pdf('/content/roteiro_viagem.md', 'roteiro_viagem.pdf')

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.005s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.009s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

## Salvar relatório final



In [ ]:
from google.colab import files
files.download('roteiro_viagem.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Visualizar e baixar relatórios específicos das tasks



In [ ]:
display_markdown(load_markdown('relatorio_local.md'))


## Guia de Viagem para Londres (23 a 30 de Novembro de 2025)

Bem-vindo a Londres! Prepare-se para uma semana inesquecível explorando a capital britânica, com foco em arte, tecnologia, vinhos e arquitetura.

### Resumo da Cidade e sua Cultura

Londres é uma metrópole vibrante com uma história rica e uma cultura diversificada. É um centro global de arte, moda, finanças e tecnologia, onde o tradicional encontra o moderno em cada esquina. A cidade abriga alguns dos museus, galerias de arte, marcos históricos e arquitetônicos mais renomados do mundo. Sua cena gastronômica é igualmente diversificada, com restaurantes que oferecem culinária de todos os cantos do globo, além de pubs tradicionais e wine bars sofisticados.

### Lugares Recomendados para Visitar e Eventos

**Arte:**

*   **Tate Modern:** Um dos maiores museus de arte moderna e contemporânea do mundo, localizado em uma antiga estação de energia. A entrada é gratuita, mas exposições especiais podem ter custo.
*   **National Gallery:** Lar de uma coleção impressionante de pinturas da Europa Ocidental do século XIII ao XIX. A entrada é gratuita.
*   **British Museum:** Explore artefatos de todo o mundo, incluindo a Pedra de Roseta e os mármores de Elgin. A entrada é gratuita.
*   **Saatchi Gallery:** Dedicada à arte contemporânea, com foco em artistas emergentes.
*   **White Cube:** Galeria de arte contemporânea com duas localizações em Londres (Bermondsey e Mason's Yard).
*   **Royal Academy of Arts:** Apresenta exposições de arte histórica e contemporânea, além de abrigar uma escola de arte.

**Tecnologia:**

*   **Science Museum:** Explore exposições interativas sobre ciência e tecnologia, incluindo a história da computação e da exploração espacial.
*   **Bletchley Park (Day Trip):** Embora um pouco fora de Londres, este é o lar histórico dos decifradores de códigos da Segunda Guerra Mundial, incluindo Alan Turing. Essencial para qualquer interessado em tecnologia e história.
*   **Silicon Roundabout (Shoreditch):** Explore a área de startups de tecnologia de Londres e talvez participe de um evento de networking.

**Arquitetura:**

*   **Tower of London:** Um castelo histórico com quase 1000 anos de história, lar das Jóias da Coroa.
*   **Houses of Parliament & Big Ben:** Admire o Palácio de Westminster e o icônico relógio Big Ben. Faça um tour para aprender sobre a história do parlamento britânico.
*   **St. Paul's Cathedral:** Uma obra-prima da arquitetura barroca inglesa, projetada por Sir Christopher Wren.
*   **The Shard:** O edifício mais alto da Europa Ocidental, oferecendo vistas panorâmicas de Londres.
*   **Sky Garden:** Um jardim público gratuito nos andares superiores de um arranha-céu com vistas deslumbrantes da cidade.
*   **Barbican Centre:** Um exemplo icônico da arquitetura brutalista, com um complexo residencial, um centro de artes e jardins.
*   **Architecture Tours:** Considere fazer um tour guiado para explorar a arquitetura histórica e contemporânea de Londres. London Architecture Tours e Architecture Foundation oferecem opções.

**Vinho:**

*   **Gordon's Wine Bar:** O wine bar mais antigo de Londres, com uma atmosfera subterrânea única.
*   **Bedales of Borough:** Um wine bar e restaurante com uma seleção diversificada de vinhos e comida deliciosa.
*   **Vinoteca:** Vários locais em Londres, oferecendo uma ampla gama de vinhos de todo o mundo.
*   **WC Clapham:** Localizado em um antigo banheiro público, este wine bar oferece uma experiência única e uma excelente seleção de vinhos.

**Eventos em Novembro:**

*   **Lord Mayor's Show (Novembro):** Um desfile anual que celebra a eleição do novo Lord Mayor de Londres, com carros alegóricos, música e fogos de artifício.

### Despesas Diárias

*   **Acomodação:** £100-£300 por noite, dependendo do hotel e localização.
*   **Alimentação:** £50-£100 por dia, dependendo das suas escolhas de restaurantes.
*   **Transporte:** £10-£20 por dia com o Oyster card ou contactless payment.
*   **Atrações:** £20-£50 por atração (muitos museus são gratuitos).

**Custo Médio Diário:** £180-£470

### Recomendações de Segurança

*   Londres é geralmente uma cidade segura, mas esteja ciente dos seus arredores, especialmente em áreas turísticas movimentadas.
*   Cuidado com batedores de carteira, principalmente em transportes públicos.
*   Não deixe seus pertences desacompanhados.
*   Se você for sair à noite, fique em áreas bem iluminadas e movimentadas.
*   Em caso de emergência, ligue para 999.

### Dicas de Costumes Locais

*   Os londrinos são geralmente educados e formais.
*   É comum dizer "por favor" e "obrigado".
*   Fazer fila é uma tradição britânica. Não fure a fila!
*   É considerado rude falar alto em transportes públicos.
*   Gorjetas são esperadas em restaurantes (10-15%) e táxis.
*   O trânsito dirige na mão esquerda. Olhe para os dois lados antes de atravessar a rua.
*   Use um "Oyster card" ou pagamento "contactless" para se locomover no transporte público para economizar dinheiro.
*   Esteja preparado para mudanças climáticas repentinas e sempre leve um casaco ou guarda-chuva.

Aproveite sua viagem a Londres!


In [ ]:
display_markdown(load_markdown('relatorio_logistica.md'))


## Guia de Viagem para Londres (23 a 30 de Novembro de 2025)

### Hospedagem

**Sugestões de Hospedagem:**

*   **The Henrietta Hotel (Covent Garden):** Hotel boutique estiloso no coração do West End, próximo a teatros, restaurantes e atrações turísticas. Ideal para quem busca uma experiência charmosa e conveniente. Estimativa de custo: £285 por noite (pesquisado em 25/05/2025 para datas futuras, pode variar).
*   **The Rockwell Hotel (Kensington):** Hotel boutique sofisticado em Kensington, próximo a museus, parques e lojas. Ideal para quem busca uma localização mais tranquila e elegante. Estimativa de custo: £250 por noite (estimativa).

**Localização Estratégica:**

Ambos os hotéis estão localizados em áreas seguras e bem conectadas por transporte público, facilitando o acesso às principais atrações de Londres. Covent Garden é ideal para quem busca estar no centro da agitação, enquanto Kensington oferece uma atmosfera mais relaxante.

### Voos

**Opções de Voos (JFK - LHR):**

*   **Companhias Aéreas:** Virgin Atlantic, American Airlines, British Airways, Delta Air Lines, JetBlue Airways (voos diretos).
*   **Estimativa de Custo:** US$424 - US$539 (ida e volta).
*   **Observação:** Os preços podem variar dependendo da antecedência da reserva, da companhia aérea e da classe tarifária. Recomenda-se pesquisar e reservar com antecedência para obter melhores preços.

### Transporte Local

**Opções de Deslocamento:**

*   **Oyster Card/Contactless Payment:** A forma mais econômica e conveniente de utilizar o transporte público em Londres (Tube, ônibus, trams, trens).
*   **Tube (Metrô):** Rápido e eficiente para se locomover entre as diferentes áreas da cidade.
*   **Ônibus:** Uma ótima opção para apreciar a paisagem e chegar a lugares onde o Tube não chega.
*   **Táxis (Black Cabs):** Mais caros, mas convenientes para traslados diretos e horários noturnos.
*   **Aplicativos de Transporte (Uber, Bolt):** Alternativa aos táxis tradicionais, com preços geralmente mais competitivos.

**Estimativa de Custos:**

*   **Transporte público:** £10-£20 por dia (Oyster card/Contactless).
*   **Táxis/Aplicativos:** Depende da distância e do trânsito, mas espere pagar £15-£30 por corrida.

### Estimativa de Custos Totais (por pessoa)

*   **Hospedagem (7 noites):** £1785 - £1995 (considerando £255-£285 por noite)
*   **Voos (ida e volta):** £340 - £430 (considerando câmbio de 1.25)
*   **Transporte (7 dias):** £70 - £140
*   **Alimentação (7 dias):** £350 - £700 (considerando £50-£100 por dia)
*   **Atrações (estimativa):** £140 - £350

**Custo Total Estimado:** £2685 - £3615

**Observações:**

*   Esta é apenas uma estimativa. Os custos reais podem variar dependendo das suas escolhas e do seu estilo de viagem.
*   Muitos museus em Londres são gratuitos, o que pode ajudar a reduzir os custos com atrações.
*   Comprar um London Pass pode ser vantajoso se você pretende visitar muitas atrações pagas.
*   Reserve voos e hospedagem com antecedência para garantir melhores preços e disponibilidade.

Aproveite sua viagem a Londres!


## Adicionar novo agente



In [ ]:
language_guide_agent = Agent(
    role="Especialista em comunicação e etiqueta local",
    goal="Gerar um guia traduzido com frases úteis, dicas de etiqueta e expressões práticas com base nas atividades do roteiro.",
    backstory=dedent(
        """
        Sou um especialista em línguas e culturas do mundo.
        Meu trabalho é ajudar turistas a se comunicarem melhor no destino, traduzindo expressões essenciais relacionadas ao roteiro,
        como pedidos em restaurantes, orientações para transporte e interações cotidianas.
        Também forneço dicas culturais para evitar gafes e tornar a experiência mais fluida e respeitosa.
        """
    ),
    llm=gemini,
    tools=[SearchTools.search_tavily],
    verbose=True,
    max_iter=5,
    allow_delegation=False,
)

In [ ]:
def language_guide_task(context, agent, destination_city):
    return Task(
        description=dedent(
            f"""
            Com base no destino ({destination_city}) e nas atividades previstas no roteiro, monte um guia traduzido com frases úteis e dicas culturais.
            Deve ser no idioma falado nesse local (ou também em inglês, caso seja uma língua aceitável para ser usada por turistas nesse local).

            Inclua expressões comuns para situações que podem ser úteis, como:
            - Interações em restaurantes (como fazer pedidos, pagar a conta, pedir recomendações);
            - Deslocamento (perguntar por rotas, chamar transporte, entender sinalização);
            - Situações cotidianas (compras, pedidos de ajuda, saudações e agradecimentos);
            - Dicas de etiqueta local que o turista deve saber (gestos, hábitos, regras sociais).

            Use linguagem clara e educativa.

            Considere o seguinte roteiro de viagem:
            '{context}'
            """
        ),
        expected_output=dedent(
            """
            Um mini-guia em português contendo:
            - Lista de frases traduzidas organizadas por situação (restaurantes, transporte, compras, etc.);
            - Dicas de etiqueta e costumes locais;
            - Sugestões de como pronunciar corretamente (quando aplicável);
            - Recomendações práticas para comunicação eficaz no destino.
            """
        ),
        agent=agent,
        output_file='guia_comunicacao.md',
    )

In [ ]:
language_guide = language_guide_task(
  [build_itinerary],
  language_guide_agent,
  destination_city,
)

### Atualizando a crew


In [ ]:
from crewai import Crew, Process
crew = Crew(
  agents=[city_info_agent, logistics_expert_agent, itinerary_planner_agent, language_guide_agent],
  tasks=[city_info, plan_logistics, build_itinerary, language_guide],
  process=Process.sequential,
  full_output=True,
  share_crew=False,
  verbose=True,
  max_rpm=15
)
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e0e2f145-eeab-471c-acee-970d2d855193                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b61b92dc-b99e-452a-aa03-43795b76d39b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b61b92dc-b99e-452a-aa03-43795b76d39b
       Status: Executing Task...
    └── 🤖 Agent: Especialista em informações da cidade
            Status: In Progress

# Agent: Especialista em informações da cidade
## Task: 
Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes locais.
Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e quaisquer atividades que se alinhem às preferências do usuário.
Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.
Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.

Viajando de: São Paulo
Para: Londres
Interesses do viajante: arte, tecnologia, vinhos, arquitetura
Chegada: 23 de novembro de 2025
Partida: 30 de novembro de 2025

Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.



🤖 Agent: Especialista em informações da cidade
    Status: In Progress

...



# Agent: Especialista em comunicação e etiqueta local
## Final Answer: 
```markdown
# Mini-Guia de Conversação e Etiqueta para sua Viagem a Londres

## Saudações e Expressões Comuns

*   **Hello:** Olá
*   **Good morning / Good afternoon / Good evening:** Bom dia / Boa tarde / Boa noite
*   **Goodbye / Bye:** Adeus / Tchau
*   **Please:** Por favor
*   **Thank you / Thanks:** Obrigado(a)
*   **You're welcome:** De nada
*   **Excuse me:** Com licença (para chamar atenção ou pedir desculpas)
*   **Sorry:** Desculpe
*   **Yes:** Sim
*   **No:** Não
*   **How are you?:** Como vai você?
*   **I'm fine, thank you:** Estou bem, obrigado(a).
*   **What's your name?:** Qual é o seu nome?
*   **My name is...:** Meu nome é...
*   **Nice to meet you:** Prazer em conhecê-lo(a)

## No Restaurante

*   **A table for one/two/etc., please:** Uma mesa para um/dois/etc., por favor.
*   **Do you have a menu in English?:** Você tem um cardápio em inglês?
*   **I'd like... / I'll have...:** Eu gostaria de

🚀 Crew: crew
├── 📋 Task: b61b92dc-b99e-452a-aa03-43795b76d39b
│      Assigned to: Especialista em informações da cidade
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em informações da cidade
│           Status: ✅ Completed
├── 📋 Task: cb25449a-98c2-4537-87c6-4cec93fbf588
│      Assigned to: Especialista em logística de viagem
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em logística de viagem
│           Status: ✅ Completed
├── 📋 Task: c0c55e71-4cfa-4d89-b755-1510dd840266
│      Assigned to: Planejador de itinerário personalizado
│      Status: ✅ Completed
│   └── 🤖 Agent: Planejador de itinerário personalizado
│           Status: ✅ Completed
└── 📋 Task: de4ffb0f-ab75-4d17-bdc6-7e9be3039c57
       Status: Executing Task...
    └── 🤖 Agent: Especialista em comunicação e etiqueta local
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b61b92dc-b99e-452a-aa03-43795b76d39b
│      Assigned to: Especialista em informações da cidade
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em informações da cidade
│           Status: ✅ Completed
├── 📋 Task: cb25449a-98c2-4537-87c6-4cec93fbf588
│      Assigned to: Especialista em logística de viagem
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em logística de viagem
│           Status: ✅ Completed
├── 📋 Task: c0c55e71-4cfa-4d89-b755-1510dd840266
│      Assigned to: Planejador de itinerário personalizado
│      Status: ✅ Completed
│   └── 🤖 Agent: Planejador de itinerário personalizado
│           Status: ✅ Completed
└── 📋 Task: de4ffb0f-ab75-4d17-bdc6-7e9be3039c57
       Assigned to: Especialista em comunicação e etiqueta local
       Status: ✅ Completed
    └── 🤖 Agent: Especialista em comunicação e etiqueta local
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: de4ffb0f-ab75-4d17-bdc6-7e9be3039c57                                                                     │
│  Agent: Especialista em comunicação e etiqueta local                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e0e2f145-eeab-471c-acee-970d2d855193                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
md = load_markdown('guia_comunicacao.md')
display_markdown(md)


# Mini-Guia de Conversação e Etiqueta para sua Viagem a Londres

## Saudações e Expressões Comuns

*   **Hello:** Olá
*   **Good morning / Good afternoon / Good evening:** Bom dia / Boa tarde / Boa noite
*   **Goodbye / Bye:** Adeus / Tchau
*   **Please:** Por favor
*   **Thank you / Thanks:** Obrigado(a)
*   **You're welcome:** De nada
*   **Excuse me:** Com licença (para chamar atenção ou pedir desculpas)
*   **Sorry:** Desculpe
*   **Yes:** Sim
*   **No:** Não
*   **How are you?:** Como vai você?
*   **I'm fine, thank you:** Estou bem, obrigado(a).
*   **What's your name?:** Qual é o seu nome?
*   **My name is...:** Meu nome é...
*   **Nice to meet you:** Prazer em conhecê-lo(a)

## No Restaurante

*   **A table for one/two/etc., please:** Uma mesa para um/dois/etc., por favor.
*   **Do you have a menu in English?:** Você tem um cardápio em inglês?
*   **I'd like... / I'll have...:** Eu gostaria de... / Eu vou querer...
*   **What do you recommend?:** O que você recomenda?
*   **What's this dish?:** O que é este prato?
*   **Is this vegetarian/vegan?:** Isto é vegetariano/vegano?
*   **I have an allergy to...:** Eu tenho alergia a...
*   **Could I have some water, please?:** Poderia me trazer água, por favor?
*   **The bill, please:** A conta, por favor.
*   **Can I pay by card?:** Posso pagar com cartão?
*   **Do you accept contactless?:** Vocês aceitam pagamento por aproximação?
*   **Keep the change:** Fique com o troco.

## Deslocamento (Transporte)

*   **Excuse me, where is...?:** Com licença, onde fica...?
*   **How do I get to...?:** Como chego a...?
*   **Which way is it to...?:** Qual o caminho para...?
*   **Is this the right bus/train for...?:** Este é o ônibus/trem certo para...?
*   **A single/return ticket to..., please:** Uma passagem só de ida/ida e volta para..., por favor.
*   **Where is the nearest tube station?:** Onde é a estação de metrô mais próxima?
*   **Can you call a taxi for me, please?:** Você pode chamar um táxi para mim, por favor?
*   **Take me to this address, please:** Leve-me a este endereço, por favor.
*   **How much is it to...?:** Quanto custa para...?
*   **Oyster card:** Cartão de transporte público recarregável.
*   **Contactless payment accepted here:** Pagamento por aproximação aceito aqui.
*   **Mind the gap:** Cuidado com o vão (aviso comum nas estações de metrô).

## Compras

*   **How much is this?:** Quanto custa isto?
*   **Do you have this in another size/color?:** Você tem isto em outro tamanho/cor?
*   **Can I try it on?:** Posso experimentar?
*   **Where are the fitting rooms?:** Onde são os provadores?
*   **I'll take it:** Eu vou levar.
*   **Can I have a bag, please?:** Poderia me dar uma sacola, por favor?
*   **Do you have a loyalty program?:** Você tem um programa de fidelidade?
*   **I'm just looking, thank you:** Estou só olhando, obrigado(a).

## Pedidos de Ajuda

*   **Excuse me, can you help me?:** Com licença, você pode me ajudar?
*   **I'm lost:** Estou perdido(a).
*   **Where is the toilet/restroom?:** Onde é o banheiro?
*   **I need a doctor:** Eu preciso de um médico.
*   **Call the police:** Chame a polícia.
*   **Is there a pharmacy nearby?:** Tem uma farmácia por perto?
*   **I've lost my phone/wallet/passport:** Eu perdi meu celular/carteira/passaporte.

## Dicas de Etiqueta e Costumes Locais

*   **Filas (Queuing):** Os britânicos são obcecados por filas. Nunca fure a fila.
*   **"Please" e "Thank You":** Use "please" e "thank you" o tempo todo. É um sinal de educação básica.
*   **Espaço Pessoal:** Evite tocar nas pessoas e respeite o espaço pessoal, especialmente no transporte público.
*   **Tom de Voz:** Fale em um tom de voz moderado, especialmente em locais públicos.
*   **Gorjetas (Tipping):**
    *   Restaurantes: 10-15% é comum se o serviço não estiver incluído na conta.
    *   Táxis: 10% é apreciado.
    *   Pubs: Não é obrigatório, mas você pode oferecer uma bebida ao bartender ("Buy you a drink?").
*   **Pontualidade:** Ser pontual é importante, especialmente para compromissos e reservas.
*   **Comunicação:**
    *   Os londrinos podem parecer reservados, mas geralmente são prestativos. Não hesite em pedir informações.
    *   Seja educado e paciente.
    *   Evite falar muito alto ao celular em público.
*   **Metro/Tube:**
    *   Fique à direita nas escadas rolantes para permitir que outros passem à esquerda.
    *   Ofereça seu assento a idosos, mulheres grávidas ou pessoas com deficiência.
*   **Visitando Casas:** Se for convidado para a casa de alguém, é educado levar um pequeno presente, como flores ou vinho.

## Pronúncia

*   A pronúncia britânica pode ser diferente do inglês americano. Algumas dicas:
    *   O "r" no final das palavras geralmente não é pronunciado (ex: "car" soa como "cah").
    *   A letra "t" pode ser pronunciada de forma mais suave, às vezes quase como um "d" entre vogais (ex: "water" soa como "wadder").
    *   A vogal "a" em palavras como "bath" e "grass" é pronunciada de forma mais aberta.
*   Não se preocupe muito com o sotaque perfeito. Os londrinos estão acostumados a ouvir diferentes sotaques e apreciarão o esforço.

## Comunicação Eficaz

*   Seja claro e conciso ao falar.
*   Não tenha medo de pedir para as pessoas repetirem se você não entender.
*   Use gestos e expressões faciais para ajudar na comunicação.
*   Tenha um aplicativo de tradução no seu celular, caso precise.
*   Relaxe e aproveite a experiência! A maioria das pessoas em Londres é amigável e disposta a ajudar.


In [ ]:
convert_md_to_pdf('guia_comunicacao.md', 'Guia de comunicação.pdf')

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.006s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.011s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

## Aplicação final e interface com Streamlit



In [ ]:
!pip install -q crewai[tools]==0.120.1 langchain-tavily duckduckgo-search weasyprint markdown2
!pip install -q streamlit python-dotenv
!npm install -q localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.6/850.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 3s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [ ]:
%%writefile .env
GOOGLE_API_KEY=#######
GROQ_API_KEY=#######
TAVILY_API_KEY=#######

Writing .env


### Código das tools (trip_tools.py)



In [ ]:
%%writefile trip_tools.py
from crewai.tools import tool
from langchain_tavily import TavilySearch
from langchain_community.tools import DuckDuckGoSearchResults

class SearchTools:
    @tool("Pesquisa na internet")
    def search_tavily(query: str = "") -> str:
        """
        Search the web using Tavily API.
        Recommended for more structured and recent information.
        """
        search_tavily = TavilySearch(max_results=4)
        search_res = search_tavily.invoke(query)
        return search_res

    @tool("Pesquisa na internet com DuckDuckGo")
    def search_duckduckgo(query: str):
        """
        Search the web using DuckDuckGo.
        Returns a list of search results.
        """
        search_tool = DuckDuckGoSearchResults(num_results=4, verbose=True)
        return search_tool.run(query)

class CalculatorTools:
    @tool("Faça um cálculo")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Erro: Sintaxe inválida"

Writing trip_tools.py


### Código dos componentes (trip_components.py)



In [ ]:
%%writefile trip_components.py
from crewai import Agent, Task, LLM
from trip_tools import SearchTools, CalculatorTools
from textwrap import dedent
import os
from dotenv import load_dotenv
load_dotenv()

class TripAgents:
    def __init__(self):
        self.gemini = LLM(
            model="gemini/gemini-2.0-flash",
            api_key=os.getenv("GOOGLE_API_KEY"),
        )

    def city_info_agent(self):
        return Agent(
            role="Especialista em informações da cidade",
            goal=dedent(
                """Reunir informações úteis sobre o destino escolhido, com base nos interesses e datas da viagem,
                ajudando viajantes a entender o contexto geral do local."""
            ),
            backstory=dedent(
                """
                Sou um especialista com amplo conhecimento sobre cidades ao redor do mundo,
                capaz de fornecer detalhes atualizados sobre clima, eventos culturais, segurança,
                costumes locais e outros aspectos práticos para quem pretende visitar o local.
                Sou apaixonado por compartilhar as melhores experiências e 'joias escondidas' do local.
                """
            ),
            llm=self.gemini,
            tools=[SearchTools.search_tavily],
            verbose=True,
            max_iter=10,
            allow_delegation=False,
      )

    def logistics_expert_agent(self):
        return Agent(
            role="Especialista em logística de viagem",
            goal="Identificar as melhores opções logísticas para a viagem, com foco em praticidade, custo-benefício e conforto.",
            backstory=dedent(
                """
                Sou um profissional focado em planejar a logística da viagem: transporte, hospedagem e deslocamento local.
                Tenho conhecimento sobre companhias aéreas, apps de mobilidade, regiões seguras para se hospedar e otimização de trajetos.
                """
            ),
            llm=self.gemini,
            tools=[
                SearchTools.search_tavily,
                CalculatorTools.calculate
            ],
            verbose=True,
            max_iter=10,
            allow_delegation=False,
        )

    def itinerary_planner_agent(self):
        return Agent(
            role="Planejador de itinerário personalizado",
            goal="Criar um roteiro completo com base nas preferências do usuário.",
            backstory=dedent(
                """
                Sou um guia profissional apaixonado por viagens, especialista em organização de itinerários personalizados.
                Minha missão é integrar dados sobre clima, atrações, eventos e logística em uma experiência otimizada para o turista.
                """
            ),
            llm=self.gemini,
            tools=[SearchTools.search_tavily],
            verbose=True,
            max_iter=10,
            allow_delegation=False,
        )

    def language_guide_agent(self):
        return Agent(
            role="Especialista em comunicação e etiqueta local",
            goal="Gerar um guia traduzido com frases úteis, dicas de etiqueta e expressões práticas com base nas atividades do roteiro.",
            backstory=dedent(
                """
                Sou um especialista em línguas e culturas do mundo.
                Meu trabalho é ajudar turistas a se comunicarem melhor no destino, traduzindo expressões essenciais relacionadas ao roteiro,
                como pedidos em restaurantes, orientações para transporte e interações cotidianas.
                Também forneço dicas culturais para evitar gafes e tornar a experiência mais fluida e respeitosa.
                """
            ),
            llm=self.gemini,
            tools=[SearchTools.search_tavily],
            verbose=True,
            max_iter=5,
            allow_delegation=False,
        )


class TripTasks:
    def city_info_task(self, agent, from_city, destination_city, interests, date_from, date_to):
        return Task(
            description=dedent(
                f"""
                Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes locais.
                Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e quaisquer atividades que se alinhem às preferências do usuário.
                Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.
                Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.

                Viajando de: {from_city}
                Para: {destination_city}
                Interesses do viajante: {interests}
                Chegada: {date_from}
                Partida: {date_to}

                Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.
                """
            ),
            expected_output=dedent(
                f"""
                Um guia detalhado (em formato markdown) em português que inclui:
                - Resumo da cidade e sua cultura;
                - Uma lista selecionada de lugares recomendados para visitar e eventos (se houver);
                - Um detalhamento das despesas diárias, como custos médios com alimentação;
                - Recomendações de segurança;
                - Dicas de costumes locais;
                """
            ),
            agent=agent,
            output_file='relatorio_local.md',
        )

    def plan_logistics_task(self, context, agent, destination_city, interests, date_from, date_to):
        return Task(
            description=dedent(
                f"""
                Planejar a logística da viagem.
                Identificar as melhores opções de hospedagem, voos e transporte local considerando:

                Destino: {destination_city}
                Data de chegada: {date_from}
                Data de partida: {date_to}
                Interesses: {interests}

                Avalie preço, localização, conveniência e segurança.
                """
            ),
            expected_output=dedent(
                f"""
                Relatório em português (em formato markdown):
                - Sugestão de hospedagens, preferencialmente com localização estratégica;
                - Opções de voos ou meios de transporte para chegada/partida;
                - Sugestões de deslocamento dentro da cidade;
                - Estimativas de custo para cada item;
                """
            ),
            context=context,
            agent=agent,
            output_file='relatorio_logistica.md',
        )

    def build_itinerary_task(self, context, agent, destination_city, interests, date_from, date_to):
        return Task(
            description=dedent(
                f"""
                Esta tarefa sintetiza todas as informações para criar o roteiro final da viagem.
                Com base nas informações dos outros agentes, desenvolva um itinerário detalhado.
                Cada dia deve conter atividades, clima, transporte, refeições e estimativa de gastos.

                Destino: {destination_city}
                Interesses: {interests}
                Data de chegada: {date_from}
                Data de partida: {date_to}
                """
            ),
            expected_output=dedent("""
                Documento em português que inclui (em formato markdown):
                - Boas vindas e apresentação da cidade em até 2 parágrafos.
                - Programação diária com sugestões de horários e atividades
                - Atrações distribuídas por região e logística
                - Clima previsto, sugestões de transporte e alimentação
                - Eventos e festivais sazonais (se houver)
                - Estimativa de gastos por dia, custo médio das despesas diárias e pontos turísticos.
                - Visão geral dos destaques da cidade com base nas recomendações do guia.
                - Outras dicas e informações adicionais para uma viagem e estadia tranquila.
                """),
            context=context,
            agent=agent,
            output_file='roteiro_viagem.md',
        )


    def language_guide_task(self, context, agent, destination_city):
        return Task(
            description=dedent(
                f"""
                Com base no destino ({destination_city}) e nas atividades previstas no roteiro, monte um guia traduzido com frases úteis e dicas culturais.
                Deve ser no idioma falado nesse local (ou também em inglês, caso seja uma língua aceitável para ser usada por turistas nesse local).

                Inclua expressões comuns para situações que podem ser úteis, como:
                - Interações em restaurantes (como fazer pedidos, pagar a conta, pedir recomendações);
                - Deslocamento (perguntar por rotas, chamar transporte, entender sinalização);
                - Situações cotidianas (compras, pedidos de ajuda, saudações e agradecimentos);
                - Dicas de etiqueta local que o turista deve saber (gestos, hábitos, regras sociais).

                Use linguagem clara e educativa.

                Considere o seguinte roteiro de viagem:
                '{context}'
                """
            ),
            expected_output=dedent(
                """
                Um mini-guia em português contendo:
                - Lista de frases traduzidas organizadas por situação (restaurantes, transporte, compras, etc.);
                - Dicas de etiqueta e costumes locais;
                - Sugestões de como pronunciar corretamente (quando aplicável);
                - Recomendações práticas para comunicação eficaz no destino.
                """
            ),
            agent=agent,
            output_file='guia_comunicacao.md',
        )

Writing trip_components.py


### Código com funções extras (trip_utils.py)



In [ ]:
%%writefile trip_utils.py
import streamlit as st
import sys
from contextlib import contextmanager
from io import StringIO
import re

class StreamlitProcessOutput:
    def __init__(self, container):
        self.container = container
        self.output_text = ""
        self.seen_lines = set()

    def clean_text(self, text):
        # Remove ANSI escape codes
        ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
        text = ansi_escape.sub('', text)

        # Remove LiteLLM debug messages
        if text.strip().startswith('LiteLLM.Info:') or text.strip().startswith('Provider List:'):
            return None

        # Clean up the formatting
        #text = text.replace('[1m', '').replace('[95m', '').replace('[92m', '').replace('[00m', '')
        return text

    def write(self, text):
        cleaned_text = self.clean_text(text)
        if cleaned_text is None:
            return

        # Split into lines and process each line
        lines = cleaned_text.split('\n')
        new_lines = []

        for line in lines:
            line = line.strip()
            if line and line not in self.seen_lines:
                self.seen_lines.add(line)
                new_lines.append(line)

        if new_lines:
            # Add the new lines to the output
            new_content = '\n'.join(new_lines)
            self.output_text = f"{self.output_text}\n{new_content}" if self.output_text else new_content

            # Update the display
            self.container.text(self.output_text)

    def flush(self):
        pass

@contextmanager
def capture_output(container):
    """Capture stdout and redirect it to a Streamlit container."""
    string_io = StringIO()
    output_handler = StreamlitProcessOutput(container)
    old_stdout = sys.stdout
    sys.stdout = output_handler
    try:
        yield string_io
    finally:
        sys.stdout = old_stdout

# Export the capture_output function
__all__ = ['capture_output']

Writing trip_utils.py


### Código principal da aplicação




In [ ]:
%%writefile app06.py
from crewai import Crew, Process
from trip_components import TripAgents, TripTasks
from trip_utils import capture_output
from textwrap import dedent
import os
import streamlit as st
import time
import markdown2
from weasyprint import HTML
import io
from datetime import datetime
import shutil

OUTPUT_DIR = os.path.join(os.getcwd(), "viagem")
os.makedirs(OUTPUT_DIR, exist_ok=True)

files = {
    "roteiro_viagem.md": "roteiro_viagem.pdf",
    "guia_comunicacao.md": "guia_comunicacao.pdf",
    "relatorio_local.md": "relatorio_local.pdf",
    "relatorio_logistica.md": "relatorio_logistica.pdf"
}

# Configurações Streamlit
st.set_page_config(page_title="Planejamento de Viagens", page_icon="🌍")
st.title("🌍 Planejamento de Viagens")

class TripCrew:
  def __init__(self, from_city, destination_city, date_from, date_to, interests):
    self.from_city = from_city
    self.destination_city = destination_city
    self.date_from = date_from
    self.date_to = date_to
    self.interests = interests

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    city_info_agent = agents.city_info_agent()
    logistics_expert_agent = agents.logistics_expert_agent()
    itinerary_planner_agent = agents.itinerary_planner_agent()
    language_guide_agent = agents.language_guide_agent()

    city_info = tasks.city_info_task(
        city_info_agent,
        self.from_city,
        self.destination_city,
        self.interests,
        self.date_from,
        self.date_to
    )

    plan_logistics = tasks.plan_logistics_task(
        [city_info],
        logistics_expert_agent,
        self.destination_city,
        self.interests,
        self.date_from,
        self.date_to
    )

    build_itinerary = tasks.build_itinerary_task(
        [city_info, plan_logistics],
        itinerary_planner_agent,
        self.destination_city,
        self.interests,
        self.date_from,
        self.date_to
    )

    language_guide = tasks.language_guide_task(
        [build_itinerary],
        language_guide_agent,
        self.destination_city
    )

    crew = Crew(
        agents=[
            city_info_agent,
            logistics_expert_agent,
            itinerary_planner_agent,
            language_guide_agent
        ],
        tasks=[city_info, plan_logistics, build_itinerary, language_guide],
        process=Process.sequential,
        full_output=True,
        max_rpm=15,
        verbose=True
    )

    return crew.kickoff()

# Função de carregamento do markdown
def load_markdown(file_path):
  try:
    with open(file_path, 'r', encoding='utf-8') as file:
      content = file.read()
      content = content.replace("```markdown", "").replace("```", "")
      return content
  except Exception as e:
    print(f"Erro ao carregar arquivo: {str(e)}")
    return None

# Função de conversão para PDF
def convert_md_to_pdf(file_md, file_pdf):
  text_md = load_markdown(file_md)
  if text_md is not None:
    html_body = markdown2.markdown(text_md)
    style_css = """
    <style>
        body {
            font-family: Arial, Helvetica, sans-serif; font-size: 12pt; line-height: 1.5;
        }
    </style>
    """
    full_html = f"<html><head>{style_css}</head><body>{html_body}</body></html>"
    HTML(string=full_html).write_pdf(file_pdf)

with st.form("trip_form"):
  st.subheader("Informe os detalhes da sua viagem")
  from_city = st.text_input("Cidade de origem:")
  destination_city = st.text_input("Cidade de destino:")
  col1, col2 = st.columns(2)
  with col1:
    date_from = st.date_input("Data de partida:")
  with col2:
    date_to = st.date_input("Data de retorno:")
  interests = st.text_area("Interesses e preferências:", placeholder = "Ex: viagem romântica, museus, gastronomia local...")
  submitted = st.form_submit_button("Gerar roteiro")

if submitted:
  date_from_str = date_from.strftime("%d de %B de %Y")
  date_to_str = date_to.strftime("%d de %B de %Y")

  if not (from_city and destination_city and date_from and date_to and interests):
    st.warning("Por favor, preencha todos os campos do formulário")
  else:
    with st.status("Montando seu roteiro... Isso pode levar alguns instantes...") as status:
      try:
        process_container = st.container(height=300, border=True)
        output_container = process_container.container()

        with capture_output(output_container) as container_content:
          trip_crew = TripCrew(from_city, destination_city, date_from_str, date_to_str, interests)
          result = trip_crew.run()
          status.update(label="✅ Roteiro gerado com sucesso!", state="complete", expanded=False)
      except Exception as e:
        status.update(label="Ocorreu um erro", state = "error")
        st.error(f"Erro: {str(e)}")
        st.stop()

    for md_file, pdf_file in files.items():
      convert_md_to_pdf(md_file, os.path.join(OUTPUT_DIR, pdf_file))

    st.toast("Arquivos PDFs salvos no diretório", icon="✅")

    for src_file in files.keys():
      if os.path.exists(src_file):
        shutil.move(src_file, os.path.join(OUTPUT_DIR, src_file))

files_md = [md for md in files if os.path.exists(os.path.join(OUTPUT_DIR, md))]
if len(files_md) == len(files):
  if not submitted:
    st.info("Exibindo relatórios da geração anterior")
  tab1, tab2, tab3, tab4 = st.tabs(["🗺️ Roteiro de Viagem", "📖 Guia de Comunicação", "📍 Relatório Cidade", "✈️ Relatório Logística"])
  with tab1:
    st.markdown(load_markdown(os.path.join(OUTPUT_DIR, "roteiro_viagem.md")))
  with tab2:
    st.markdown(load_markdown(os.path.join(OUTPUT_DIR, "guia_comunicacao.md")))
  with tab3:
    st.markdown(load_markdown(os.path.join(OUTPUT_DIR, "relatorio_local.md")))
  with tab4:
    st.markdown(load_markdown(os.path.join(OUTPUT_DIR, "relatorio_logistica.md")))

Writing app06.py


In [ ]:
!streamlit run app06.py &>/content/logs.txt &

!wget -q -O - ipv4.icanhazip.com
!npx localtunnel --port 8501

34.138.218.13
⠙⠹your url is: https://heavy-seals-cheer.loca.lt
^C


> **Importante:** Caso o comando acima com localtunnel não funcione, use o código abaixo (Para mais detalhes, consulte a aula "Aviso sobre uso no Colab" da seção 2)

### Alternativa com ngrok

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

!ngrok config add-authtoken SEU_TOKEN_AQUI
!streamlit run app06.py --server.port 8501 &>/content/logs.txt &

public_url = ngrok.connect(8501)
public_url